##  Create higher-quality recommendations in your e-commerce platform  with Amazon Personalize

This notebook is part of Amazon re:Invent 2019 Builder's Session "RET310 -  Create higher-quality recommendations in your e-commerce platform".

In this notebook, you will be able to run & deploy on your AWS account all steps taken during the session. By the end of it, endpoints will be ready to be used to perform recommendations on your e-commerce application.

More details about the Buider's Session at this link: https://www.portal.reinvent.awsevents.com/connect/sessionDetail.ww?SESSION_ID=98681

The dataset used on this exercise was created by Olist and made available on Kaggle platform. To download the latest version of this dataset, as also to check details about the data and their schemas, please check: https://www.kaggle.com/olistbr/brazilian-ecommerce 

#### Important Note

As for all Machine Learning solutions, data preparation is a key step to achieve higher quality on the final solution. For the data preparation steps taken on this exercise, please check the notebook "RET310 - Data preparation steps.ipynb" also present on this repository.

## Table of Contents

* [Section 1 - Preparation steps](#first-section)
* [Section 2 - Data schemas creation steps](#second-section)
* [Section 3 - Create dataset group and datasets](#third-section)
* [Section 4 - Importing Olist data into the datasets](#forth-section)
* [Section 5 - Creating Solutions and training new Solutions versions](#fifth-section)
* [Section 6 - Deploying Personalize Campaigns with the trained Solutions](#sixth-section)
* [Section 7 - Testing the deployed Campaigns using the Python SDK](#seventh-section)
* [Section 8 - Clean Up steps](#eighth-section)

### Section 1 - Preparation steps <a class="anchor" id="first-section"></a>

On this section you will perform the pre-requisites to deploy the Amazon Personalize solution. It includes importing Python modules, defining S3 bucket information and creating IAM roles with appropriate access.

In [5]:
import json
import time
import boto3

import pandas as pd

print(boto3.__version__)

personalize = boto3.client('personalize')

1.10.44


At this point, the IAM Role to be used by Amazon Personalize will be created:

In [6]:
iam = boto3.client('iam')

path='/'
role_name='ret310-personalize-role' # you may change this role name if needed
description='IAM role with permissions to run the lab RET310'
trust_policy={
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "",
      "Effect": "Allow",
      "Principal": {
        "Service": "personalize.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

try:
    response = iam.create_role(
        Path=path,
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description=description,
        MaxSessionDuration=3600
    )
except Exception as e:
    print(e)
    
roleArn = response['Role']['Arn']

print(roleArn)

arn:aws:iam::807802666179:role/ret310-personalize-role


With the IAM Role created, an IAM Policy is created and attached to the IAM Role, allowing the appropriate access to the required services:

In [7]:
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:ListBucket",
                "s3:GetObject",
                "personalize:*"
            ],
            "Resource": "*"
        }
    ]
}

response = iam.create_policy(PolicyName='ret310-policy', PolicyDocument=json.dumps(policy))
policyArn = response['Policy']['Arn']

print(policyArn)

iam.attach_role_policy(RoleName=role_name, PolicyArn=policyArn)

arn:aws:iam::807802666179:policy/ret310-policy


{'ResponseMetadata': {'RequestId': '0bf70ba2-2b19-4590-9802-a2731706be84',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0bf70ba2-2b19-4590-9802-a2731706be84',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Sat, 21 Dec 2019 03:20:25 GMT'},
  'RetryAttempts': 0}}

Create S3 bucket and upload data files to S3:

In [ ]:
region = 'ap-southeast-2'
bucket_name = 'personalize-wei-test'

data_dir = './data/'

users_data = 'users-olist.csv'
products_data = 'products-olist.csv'
interactions_data = 'orderItems-olist.csv'

In [68]:
df_users = pd.read_csv(data_dir + users_data)
df_users

,USER_ID,ZipCode,State
0,861eff4711a542e4b93843c6dd7febb0,14409,SP
1,290c77bc529b7ac935b93aa66c333dc3,9790,SP
2,060e732b5b29e8181a18229c7b0b2b5e,1151,SP
3,259dac757896d24d7702b9acbbff3f3c,8775,SP
4,345ecd01c38d18a9036ed96c73b8d066,13056,SP
...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,3937,SP
96092,d52a67c98be1cf6a5c84435bd38d095d,6764,SP
96093,e9f50caf99f032f0bf3c55141f019d99,60115,CE
96094,73c2643a0a458b49f58cea58833b192e,92120,RS


In [57]:
df_products = pd.read_csv(data_dir + products_data)
df_products

,ITEM_ID,CATEGORY
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,art
2,96bd76ec8810374ed1b65e291975717f,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,baby
4,9dc1a7de274444849c219cff195d0b71,housewares
...,...,...
32323,a0b7d5a992ccda646f2d34e418fff5a0,furniture_decor
32324,bf4538d88321d0fd4412a93c974510e6,construction_tools_lights
32325,9a7c6041fa9592d9d9ef6cfe62a71f8c,bed_bath_table
32326,83808703fc0706a22e264b9d75f04a2e,computers_accessories


In [69]:
df_interactions = pd.read_csv(data_dir + interactions_data)
df_interactions

,USER_ID,ITEM_ID,Timestamp
0,9ef432eb6251297304e76186b10a928d,87285b34884572647811a353c7ac498a,1506941793
1,b0830fb4747a6c6d20dea0b8c802d7ef,595fac2a385ac33a80bd5114aec74eb8,1532464897
2,41ce2a54c0b03bf3443c3d931a367089,aa4383b373c6aca5d8797843e5594415,1533717529
3,f88197465ea7920adcdbec7375364d82,d0b61bfb1de832b15ba9d266ca96e5b0,1511033286
4,8ab97904e6daea8866dbdbc4fb7aad2c,65266b2da20d04dbe00c5c2d3bb7859e,1518556719
...,...,...,...
101956,39bd1228ee8140590ac3aca26f2dfe00,ac35486adb7b02598c182c2ff2e05254,1489053245
101957,1fca14ff2861355f6e5f14306ff977a7,f1d4ce8c6dd66c47bbaa8c6781c2a923,1517921938
101958,1aa71eb042121263aafbe80c1b562c9c,b80910977a37536adeddd63663f916ad,1503845203
101959,b331b74b18dc79bcdf6532d51e1637c1,d1c427060a0f73f6b889a5c7c61f2ac4,1515446907


In [40]:
# Create S3 bucket and upload data
from botocore.exceptions import ClientError

s3_client = boto3.client('s3', region_name=region)

In [35]:
try:
    s3_client.create_bucket(Bucket=bucket_name,
                            CreateBucketConfiguration={'LocationConstraint': region})
except ClientError as e:
    print(e)

for file_name in [users_data, products_data, interactions_data]:
    with open(data_dir + file_name, 'rb') as f:
        s3_client.upload_fileobj(f, bucket_name, file_name)
        print(f'Uploaded s3://{bucket_name}/{file_name}')

An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.
Uploaded s3://personalize-wei-test/users-olist.csv
Uploaded s3://personalize-wei-test/products-olist.csv
Uploaded s3://personalize-wei-test/orderItems-olist.csv


In [43]:
bucket_policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{bucket_name}",
                f"arn:aws:s3:::{bucket_name}/*"
            ]
        }
    ]
}

s3_client.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(bucket_policy))

{'ResponseMetadata': {'RequestId': '2F76244CD1AB2A23',
  'HostId': 'Iy6rpPdTbiy7vUAN7OslTrKdFpzPo8lMgkdRw+4imPNGxG7ZN65l7L3drh+dgX0rQGoshU7IpGE=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'Iy6rpPdTbiy7vUAN7OslTrKdFpzPo8lMgkdRw+4imPNGxG7ZN65l7L3drh+dgX0rQGoshU7IpGE=',
   'x-amz-request-id': '2F76244CD1AB2A23',
   'date': 'Sat, 21 Dec 2019 05:10:26 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Section 2 - Data schemas creation steps <a class="anchor" id="second-section"></a>

At this point, the data stored on S3 will be accessed, as also the initial Amazon Personalize preparation will be done, including data schemas

In [66]:
# df_products[df_products['ITEM_ID'].isin([
#     '99a4788cb24856965c36a24e339b6058',
#     'aca2eb7d00ea1a7b8ebd4e68314663af',
#     '422879e10f46682990de24d770e7f83d',
#     'd1c427060a0f73f6b889a5c7c61f2ac4',
#     '389d119b48cf3043d311335e499d9c6b',
#     '53b36df67ebb7c41585e8d54d6772e08',
#     '368c6c730842d78016ad823897a372db',
#     '53759a2ecddad2bb87a079a1f1519f73',
#     '154e7e31ebfa092203795c972e5804a6',
#     '2b4609f8948be18874494203496bc318'
# ])]


from functools import reduce

reduce(pd.DataFrame.append, map(lambda i: df_products[df_products['ITEM_ID'] == i], [
    '99a4788cb24856965c36a24e339b6058',
    'aca2eb7d00ea1a7b8ebd4e68314663af',
    '422879e10f46682990de24d770e7f83d',
    'd1c427060a0f73f6b889a5c7c61f2ac4',
    '389d119b48cf3043d311335e499d9c6b',
    '53b36df67ebb7c41585e8d54d6772e08',
    '368c6c730842d78016ad823897a372db',
    '53759a2ecddad2bb87a079a1f1519f73',
    '154e7e31ebfa092203795c972e5804a6',
    '2b4609f8948be18874494203496bc318'    
]))


,ITEM_ID,CATEGORY
9475,99a4788cb24856965c36a24e339b6058,bed_bath_table
13167,aca2eb7d00ea1a7b8ebd4e68314663af,furniture_decor
13776,422879e10f46682990de24d770e7f83d,garden_tools
8122,d1c427060a0f73f6b889a5c7c61f2ac4,computers_accessories
29724,389d119b48cf3043d311335e499d9c6b,garden_tools
768,53b36df67ebb7c41585e8d54d6772e08,watches_gifts
4499,368c6c730842d78016ad823897a372db,garden_tools
31491,53759a2ecddad2bb87a079a1f1519f73,garden_tools
1700,154e7e31ebfa092203795c972e5804a6,health_beauty
5702,2b4609f8948be18874494203496bc318,health_beauty


Here the Amazon Personalize schemas are created, based on Apache Avro standard. For more details, check: https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html

In [17]:
users_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID", # required field
            "type": "string"
        },
        {
            "name": "ZipCode", # +1 metadata field
            "type": "long"
        },
        {
            "name": "State",
            "type": "string"
        }
    ],
    "version": "1.0"
}

products_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID", # required field
            "type": "string"
        },
        {
            "name": "CATEGORY", # +1 metadata field
            "type": "string",
            "categorical": True
        }
    ],
    "version": "1.0"
}

interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID", # required field
            "type": "string"
        },
        {
            "name": "ITEM_ID", # required field
            "type": "string"
        },
        {
            "name": "Timestamp", # timestamp
            "type": "long"
        }
    ],
    "version": "1.0"
}


'{"type": "record", "name": "Users", "namespace": "com.amazonaws.personalize.schema", "fields": [{"name": "USER_ID", "type": "string"}, {"name": "ZipCode", "type": "long"}, {"name": "State", "type": "string"}], "version": "1.0"}'

With the Avro schemas defined, the schemas will be created for Users, Products and Interactions data:

In [18]:
create_users_schema_response = personalize.create_schema(name = 'ret310-users-schema', 
                                                         schema = json.dumps(users_schema))
create_products_schema_response = personalize.create_schema(name = 'ret310-products-schema', 
                                                            schema = json.dumps(products_schema))
create_interactions_schema_response = personalize.create_schema(name = 'ret310-interactions-schema', 
                                                                schema = json.dumps(interactions_schema))

users_schema_arn = create_users_schema_response['schemaArn']
print(users_schema_arn)

products_schema_arn = create_products_schema_response['schemaArn']
print(products_schema_arn)

interactions_schema_arn = create_interactions_schema_response['schemaArn']
print(interactions_schema_arn)


arn:aws:personalize:ap-southeast-2:807802666179:schema/ret310-users-schema
arn:aws:personalize:ap-southeast-2:807802666179:schema/ret310-products-schema
arn:aws:personalize:ap-southeast-2:807802666179:schema/ret310-interactions-schema


### Section 3 - Create dataset group and datasets <a class="anchor" id="third-section"></a>

First you will create the dataset group to be used on this exercise - it will be called "ret310-dataset-group":

In [19]:
create_dataset_group_response = personalize.create_dataset_group(name = 'ret310-dataset-group')
print(create_dataset_group_response)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(datasetGroupArn = dataset_group_arn)
    status = describe_dataset_group_response['datasetGroup']['status']
    print('DatasetGroup: {}'.format(status))
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
        
    time.sleep(60)

{'datasetGroupArn': 'arn:aws:personalize:ap-southeast-2:807802666179:dataset-group/ret310-dataset-group', 'ResponseMetadata': {'RequestId': 'cc774ed4-10a1-4b87-8e3f-c5c1657d611e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Sat, 21 Dec 2019 04:08:41 GMT', 'x-amzn-requestid': 'cc774ed4-10a1-4b87-8e3f-c5c1657d611e', 'content-length': '104', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


With the dataset group created, now you will create one dataset for each type of data: Users, Products and Interactions:

In [20]:
%%time

dataset_type = 'USERS'
create_dataset_response = personalize.create_dataset(name = 'ret310-users-dataset',
                                                     datasetType = dataset_type,
                                                     datasetGroupArn = dataset_group_arn,
                                                     schemaArn = users_schema_arn)

users_dataset_arn = create_dataset_response['datasetArn']
print(users_dataset_arn)

dataset_type = 'ITEMS'
create_dataset_response = personalize.create_dataset(name = 'ret310-items-dataset',
                                                     datasetType = dataset_type,
                                                     datasetGroupArn = dataset_group_arn,
                                                     schemaArn = products_schema_arn)

products_dataset_arn = create_dataset_response['datasetArn']
print(products_dataset_arn)

dataset_type = 'INTERACTIONS'
create_dataset_response = personalize.create_dataset(name = 'ret310-interactions-dataset',
                                                     datasetType = dataset_type,
                                                     datasetGroupArn = dataset_group_arn,
                                                     schemaArn = interactions_schema_arn)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(create_dataset_response)

arn:aws:personalize:ap-southeast-2:807802666179:dataset/ret310-dataset-group/USERS
arn:aws:personalize:ap-southeast-2:807802666179:dataset/ret310-dataset-group/ITEMS
{'datasetArn': 'arn:aws:personalize:ap-southeast-2:807802666179:dataset/ret310-dataset-group/INTERACTIONS', 'ResponseMetadata': {'RequestId': '68a265db-5b7c-47e4-99e2-45c335b5dfff', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Sat, 21 Dec 2019 04:13:23 GMT', 'x-amzn-requestid': '68a265db-5b7c-47e4-99e2-45c335b5dfff', 'content-length': '106', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
CPU times: user 22.9 ms, sys: 4.22 ms, total: 27.1 ms
Wall time: 348 ms


### Section 4 - Importing Olist data into the datasets<a class="anchor" id="forth-section"></a>

Now with all the datasets properly created, you will import the Olist data, stored on S3, into them.

First you will import the Users data:

In [45]:
%%time

create_users_import_job_response = personalize.create_dataset_import_job(jobName = 'ret310-users-import-job',
                                                                         datasetArn = users_dataset_arn,
                                                                         roleArn = roleArn,
                                                                         dataSource = {
                                                                             'dataLocation': 's3://{}/{}'.format(bucket_name, users_data)
                                                                         })

dataset_import_job_arn = create_users_import_job_response['datasetImportJobArn']

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    create_users_import_job_response = personalize.describe_dataset_import_job(datasetImportJobArn = dataset_import_job_arn)
    status = create_users_import_job_response['datasetImportJob']['status']
    print('DatasetImportJob: {}'.format(status))
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 107 ms, sys: 19.3 ms, total: 127 ms
Wall time: 12min 6s


Then you will import the Products data:

In [46]:
%%time

create_products_import_job_response = personalize.create_dataset_import_job(jobName = 'ret310-items-import-job',
                                                                            datasetArn = products_dataset_arn,
                                                                            roleArn = roleArn,
                                                                            dataSource = {
                                                                                'dataLocation': 's3://{}/{}'.format(bucket_name, products_data)
                                                                            })

dataset_import_job_arn = create_products_import_job_response['datasetImportJobArn']

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    create_products_import_job_response = personalize.describe_dataset_import_job(datasetImportJobArn = dataset_import_job_arn)
    status = create_products_import_job_response['datasetImportJob']['status']
    print('DatasetImportJob: {}'.format(status))
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 90.1 ms, sys: 17.7 ms, total: 108 ms
Wall time: 12min 2s


And then you will finish the importing process, with the Interactions data import:

In [47]:
%%time

create_interactions_import_job_response = personalize.create_dataset_import_job(jobName = 'ret310-interactions-import-job',
                                                                                datasetArn = interactions_dataset_arn,
                                                                                roleArn = roleArn,
                                                                                dataSource = {
                                                                                    'dataLocation': 's3://{}/{}'.format(bucket_name, interactions_data)
                                                                                })

dataset_import_job_arn = create_interactions_import_job_response['datasetImportJobArn']

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    create_interactions_import_job_response = personalize.describe_dataset_import_job(datasetImportJobArn = dataset_import_job_arn)
    status = create_interactions_import_job_response['datasetImportJob']['status']
    print('DatasetImportJob: {}'.format(status))
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 153 ms, sys: 23.1 ms, total: 176 ms
Wall time: 12min 20s


### Section 5 - Creating Solutions and training new Solutions versions<a class="anchor" id="fifth-section"></a>

Now with all data ready, you will start creating and training Personalize Solutions with them - For this exercise, you will use Solutions based on the following recipes:
- Popularity
- Item-to-Item similarity (SIMS)
- Personalized Ranking

For more details about the Personalize predefined Recipes, check: https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html

First you will list all the available recipes:

In [48]:
list_recipes_response = personalize.list_recipes()
for recipe in list_recipes_response['recipes']:
    print(recipe['name'], '-', recipe['recipeArn'], '-', recipe['status'])

aws-hrnn - arn:aws:personalize:::recipe/aws-hrnn - ACTIVE
aws-hrnn-coldstart - arn:aws:personalize:::recipe/aws-hrnn-coldstart - ACTIVE
aws-hrnn-metadata - arn:aws:personalize:::recipe/aws-hrnn-metadata - ACTIVE
aws-personalized-ranking - arn:aws:personalize:::recipe/aws-personalized-ranking - ACTIVE
aws-popularity-count - arn:aws:personalize:::recipe/aws-popularity-count - ACTIVE
aws-sims - arn:aws:personalize:::recipe/aws-sims - ACTIVE


Then you will create the solutions to be used by this exercise into the "ret310-dataset-group":

In [49]:
popularity_arn = 'arn:aws:personalize:::recipe/aws-popularity-count'
sims_arn = 'arn:aws:personalize:::recipe/aws-sims'
ranking_arn = 'arn:aws:personalize:::recipe/aws-personalized-ranking'

create_solution_response = personalize.create_solution(name = 'ret310-popularity-solution',
                                                       datasetGroupArn = dataset_group_arn,
                                                       recipeArn = popularity_arn)
popularity_solution_arn = create_solution_response['solutionArn']
print(popularity_solution_arn)

create_solution_response = personalize.create_solution(name = 'ret310-sims-solution',
                                                       datasetGroupArn = dataset_group_arn,
                                                       recipeArn = sims_arn)
sims_solution_arn = create_solution_response['solutionArn']
print(sims_solution_arn)

create_solution_response = personalize.create_solution(name='ret310-sims-hpo-solution',
                                                       recipeArn = sims_arn,
                                                       datasetGroupArn = dataset_group_arn,
                                                       performHPO=True,
                                                       performAutoML=False,
                                                       solutionConfig={
                                                                        "hpoConfig": {
                                                                            "algorithmHyperParameterRanges": {
                                                                                "categoricalHyperParameterRanges": [],
                                                                                "continuousHyperParameterRanges": [
                                                                                    {
                                                                                        "name": "popularity_discount_factor",
                                                                                        "minValue": 0,
                                                                                        "maxValue": 1
                                                                                    }
                                                                                ],
                                                                                "integerHyperParameterRanges": [
                                                                                    {
                                                                                        "name": "min_cointeraction_count",
                                                                                        "minValue": 0,
                                                                                        "maxValue": 10
                                                                                    }
                                                                                ]
                                                                            },
                                                                            "hpoResourceConfig": {
                                                                                "maxNumberOfTrainingJobs": "20",
                                                                                "maxParallelTrainingJobs": "5"
                                                                            }
                                                                        },
                                                                        "featureTransformationParameters": {
                                                                            "max_item_interaction_count_percentile": "0.9",
                                                                            "max_user_history_length_percentile": "0.995",
                                                                            "min_item_interaction_count_percentile": "0.01",
                                                                            "min_user_history_length_percentile": "0.005"
                                                                        },
                                                                        "algorithmHyperParameters": {
                                                                            "min_cointeraction_count": "3",
                                                                            "popularity_discount_factor": "0.5"
                                                                        },
                                                                    }
                                                      )
sims_hpo_solution_arn = create_solution_response['solutionArn']
print(sims_hpo_solution_arn)

create_solution_response = personalize.create_solution(name = 'ret310-ranking-solution',
                                                       datasetGroupArn = dataset_group_arn,
                                                       recipeArn = ranking_arn)
ranking_solution_arn = create_solution_response['solutionArn']
print(ranking_solution_arn)

arn:aws:personalize:ap-southeast-2:807802666179:solution/ret310-popularity-solution
arn:aws:personalize:ap-southeast-2:807802666179:solution/ret310-sims-solution
arn:aws:personalize:ap-southeast-2:807802666179:solution/ret310-sims-hpo-solution
arn:aws:personalize:ap-southeast-2:807802666179:solution/ret310-ranking-solution


With all the solutions created, it is time to train them using the Olist dataset - First you will train the solution based on the Popularity - This one will be used just as a quality baseline, when comapring with SIMS based solutions:

In [50]:
%%time

create_solution_version_response = personalize.create_solution_version(solutionArn = popularity_solution_arn)
popularity_version_arn = create_solution_version_response['solutionVersionArn']

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(solutionVersionArn = popularity_version_arn)
    status = describe_solution_version_response['solutionVersion']['status']
    print('SolutionVersion: {}'.format(status))
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
        
    time.sleep(60)

get_solution_metrics_response = personalize.get_solution_metrics(solutionVersionArn = popularity_version_arn)
print(json.dumps(get_solution_metrics_response['metrics'], indent=2))

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

Now you will create the first solution using SIMS recipe - it will use the standard parameters while training:

In [51]:
%%time

create_solution_version_response = personalize.create_solution_version(solutionArn = sims_solution_arn)
sims_version_arn = create_solution_version_response['solutionVersionArn']

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(solutionVersionArn = sims_version_arn)
    status = describe_solution_version_response['solutionVersion']['status']
    print('SolutionVersion: {}'.format(status))
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
        
    time.sleep(60)

get_solution_metrics_response = personalize.get_solution_metrics(solutionVersionArn = sims_version_arn)
print(json.dumps(get_solution_metrics_response['metrics'], indent=2))

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

Now you will create another SIMS based solution, but this one will use the Hyperparameters Optimization (HPO) feature from Amazon Personalize. With HPO, Personalize will automatically tune the Solution and will provide you a final solution version using the best parameters from the training - based on the training metrics.

For more details about Amazon Personalize HPO, please check: https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html

In [52]:
%%time

create_solution_version_response = personalize.create_solution_version(solutionArn = sims_hpo_solution_arn)
sims_hpo_version_arn = create_solution_version_response['solutionVersionArn']

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(solutionVersionArn = sims_hpo_version_arn)
    status = describe_solution_version_response['solutionVersion']['status']
    print('SolutionVersion: {}'.format(status))
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
        
    time.sleep(60)

get_solution_metrics_response = personalize.get_solution_metrics(solutionVersionArn = sims_hpo_version_arn)
print(json.dumps(get_solution_metrics_response['metrics'], indent=2))

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

And the last Solution you will train is based on personalized ranking recipe. Instead of recommending products to a user as the last ones trained, it will prioritize a given list of products to a specific user. It is extremely useful to guide in which order is more adequate to present products to a given customer (like per example, when building a dynamic carousel on an e-commerce):

In [53]:
%%time

create_solution_version_response = personalize.create_solution_version(solutionArn = ranking_solution_arn)
ranking_version_arn = create_solution_version_response['solutionVersionArn']

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(solutionVersionArn = ranking_version_arn)
    status = describe_solution_version_response['solutionVersion']['status']
    print('SolutionVersion: {}'.format(status))
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
        
    time.sleep(60)

get_solution_metrics_response = personalize.get_solution_metrics(solutionVersionArn = ranking_version_arn)
print(json.dumps(get_solution_metrics_response['metrics'], indent=2))

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

### Section 6 - Deploying Personalize Campaigns with the trained Solutions<a class="anchor" id="sixth-section"></a>

Deploying Campaigns is the way you can start doing actual recommendations with Personalize. With a Campaign, you will have an endpoint, backed by the Solution version you chose.

For more details about Amazon Personalize Campaigns, please check: https://docs.aws.amazon.com/personalize/latest/dg/campaigns.html

Here you will first create the campaign for the solution based on SIMS recipe -- here without the HPO optimization:

In [54]:
%%time

create_campaign_response = personalize.create_campaign(name = 'ret310-sims-campaign',
                                                       solutionVersionArn = sims_version_arn,
                                                       minProvisionedTPS = 1)

sims_campaign_arn = create_campaign_response['campaignArn']
print(sims_campaign_arn)

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(campaignArn = sims_campaign_arn)
    status = describe_campaign_response['campaign']['status']
    print('Campaign: {}'.format(status))
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
        
    time.sleep(60)

arn:aws:personalize:ap-southeast-2:807802666179:campaign/ret310-sims-campaign
Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE
CPU times: user 90.9 ms, sys: 19.3 ms, total: 110 ms
Wall time: 9min 3s


Then you will create the campaign for the SIMS based Solution where you used the HPO optimization:

In [55]:
%%time

create_campaign_response = personalize.create_campaign(name = 'ret310-sims-hpo-campaign',
                                                       solutionVersionArn = sims_hpo_version_arn,
                                                       minProvisionedTPS = 1)

sims_hpo_campaign_arn = create_campaign_response['campaignArn']

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(campaignArn = sims_hpo_campaign_arn)
    status = describe_campaign_response['campaign']['status']
    print('Campaign: {}'.format(status))
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE
CPU times: user 90.7 ms, sys: 18.4 ms, total: 109 ms
Wall time: 10min 3s


Then you will create the campaign for the personalized ranking based solution:

In [56]:
%%time

create_campaign_response = personalize.create_campaign(name = 'ret310-ranking-campaign',
                                                       solutionVersionArn = ranking_version_arn,
                                                       minProvisionedTPS = 1)

ranking_campaign_arn = create_campaign_response['campaignArn']

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(campaignArn = ranking_campaign_arn)
    status = describe_campaign_response['campaign']['status']
    print('Campaign: {}'.format(status))
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE
CPU times: user 175 ms, sys: 28.4 ms, total: 203 ms
Wall time: 9min 18s


### Section 7 - Testing the deployed Campaigns using the Python SDK<a class="anchor" id="seventh-section"></a>

As last step on this exercise we will simualte inferences against the Amazon Personalize Campaigns endpoints.

First we will use the get_recommendations() API against the SIMS based Solution:

In [77]:
personalize_runtime = boto3.client('personalize-runtime')

response = personalize_runtime.get_recommendations(campaignArn = sims_hpo_campaign_arn,
                                                   itemId = 'cef67bcfe19066a932b7673e239eb23d',
                                                   numResults = 10)

print(json.dumps(response, indent=2))

for item in response['itemList']:
    print('item:', item)

{
  "ResponseMetadata": {
    "RequestId": "181300ef-eec7-4cf8-8fdd-20e28bb1cbbc",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/json",
      "date": "Sat, 21 Dec 2019 11:40:51 GMT",
      "x-amzn-requestid": "181300ef-eec7-4cf8-8fdd-20e28bb1cbbc",
      "content-length": "474",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  },
  "itemList": [
    {
      "itemId": "99a4788cb24856965c36a24e339b6058"
    },
    {
      "itemId": "aca2eb7d00ea1a7b8ebd4e68314663af"
    },
    {
      "itemId": "422879e10f46682990de24d770e7f83d"
    },
    {
      "itemId": "d1c427060a0f73f6b889a5c7c61f2ac4"
    },
    {
      "itemId": "389d119b48cf3043d311335e499d9c6b"
    },
    {
      "itemId": "53b36df67ebb7c41585e8d54d6772e08"
    },
    {
      "itemId": "368c6c730842d78016ad823897a372db"
    },
    {
      "itemId": "53759a2ecddad2bb87a079a1f1519f73"
    },
    {
      "itemId": "154e7e31ebfa092203795c972e5804a6"
    },
    {
      "itemId": "

In [84]:
# isin will reorder the result, so we will use a different method to maintain the order
# df_products[df_products['ITEM_ID'].isin([
#     '99a4788cb24856965c36a24e339b6058',
#     'aca2eb7d00ea1a7b8ebd4e68314663af',
#     '422879e10f46682990de24d770e7f83d',
#     'd1c427060a0f73f6b889a5c7c61f2ac4',
#     '389d119b48cf3043d311335e499d9c6b',
#     '53b36df67ebb7c41585e8d54d6772e08',
#     '368c6c730842d78016ad823897a372db',
#     '53759a2ecddad2bb87a079a1f1519f73',
#     '154e7e31ebfa092203795c972e5804a6',
#     '2b4609f8948be18874494203496bc318'
# ])]

from functools import reduce

reduce(pd.DataFrame.append, map(lambda i: df_products[df_products['ITEM_ID'] == i], 
                                list(map(lambda i: i['itemId'], response['itemList']))
                               ))

,ITEM_ID,CATEGORY
9475,99a4788cb24856965c36a24e339b6058,bed_bath_table
13167,aca2eb7d00ea1a7b8ebd4e68314663af,furniture_decor
13776,422879e10f46682990de24d770e7f83d,garden_tools
8122,d1c427060a0f73f6b889a5c7c61f2ac4,computers_accessories
29724,389d119b48cf3043d311335e499d9c6b,garden_tools
768,53b36df67ebb7c41585e8d54d6772e08,watches_gifts
4499,368c6c730842d78016ad823897a372db,garden_tools
31491,53759a2ecddad2bb87a079a1f1519f73,garden_tools
1700,154e7e31ebfa092203795c972e5804a6,health_beauty
5702,2b4609f8948be18874494203496bc318,health_beauty


Then we will use the Personalize Ranking:

In [67]:
personalize_runtime = boto3.client('personalize-runtime')

list_of_products = ['99a4788cb24856965c36a24e339b6058', 'aca2eb7d00ea1a7b8ebd4e68314663af',
                    '422879e10f46682990de24d770e7f83d', 'd1c427060a0f73f6b889a5c7c61f2ac4',
                    '53b36df67ebb7c41585e8d54d6772e08', '389d119b48cf3043d311335e499d9c6b',
                    '368c6c730842d78016ad823897a372db', '53759a2ecddad2bb87a079a1f1519f73',
                    '154e7e31ebfa092203795c972e5804a6', '2b4609f8948be18874494203496bc318']

print('initial products list:\n', list_of_products, '\n')

response = personalize_runtime.get_personalized_ranking(campaignArn = ranking_campaign_arn,
                                                userId = '18955e83d337fd6b2def6b18a428ac77',
                                                inputList = list_of_products)

print('ranked products list:\n')

for item in response['personalizedRanking']:
    print (item['itemId'])
print()

initial products list:
 ['99a4788cb24856965c36a24e339b6058', 'aca2eb7d00ea1a7b8ebd4e68314663af', '422879e10f46682990de24d770e7f83d', 'd1c427060a0f73f6b889a5c7c61f2ac4', '53b36df67ebb7c41585e8d54d6772e08', '389d119b48cf3043d311335e499d9c6b', '368c6c730842d78016ad823897a372db', '53759a2ecddad2bb87a079a1f1519f73', '154e7e31ebfa092203795c972e5804a6', '2b4609f8948be18874494203496bc318'] 

ranked products list:

53759a2ecddad2bb87a079a1f1519f73
389d119b48cf3043d311335e499d9c6b
aca2eb7d00ea1a7b8ebd4e68314663af
154e7e31ebfa092203795c972e5804a6
2b4609f8948be18874494203496bc318
368c6c730842d78016ad823897a372db
d1c427060a0f73f6b889a5c7c61f2ac4
53b36df67ebb7c41585e8d54d6772e08
99a4788cb24856965c36a24e339b6058
422879e10f46682990de24d770e7f83d



### Section 8 - Clean Up steps<a class="anchor" id="eighth-section"></a>

After running all steps mentioned here, it is time to clean up your account. This will remove all objects/services created to make this exercise possible. It includes IAM and Personalize resources.

In [ ]:
# Removing all the deployed Campaigns - the output of this cell must be []:

personalize.delete_campaign(campaignArn=sims_campaign_arn)
personalize.delete_campaign(campaignArn=sims_hpo_campaign_arn)
personalize.delete_campaign(campaignArn=ranking_campaign_arn)
time.sleep(300)

print(personalize.list_campaigns(solutionArn=sims_version_arn)['campaigns'])
print(personalize.list_campaigns(solutionArn=sims_hpo_version_arn)['campaigns'])
print(personalize.list_campaigns(solutionArn=ranking_version_arn)['campaigns'])

In [ ]:
# Removing all the deployed Solutions versions - the output of this cell must be []:

personalize.delete_solution(solutionArn=popularity_solution_arn)
personalize.delete_solution(solutionArn=sims_solution_arn)
personalize.delete_solution(solutionArn=sims_hpo_solution_arn)
personalize.delete_solution(solutionArn=ranking_solution_arn)
time.sleep(300)

personalize.list_solutions(datasetGroupArn=dataset_group_arn)['solutions']

In [ ]:
# Removing all datasets - the output of this cell must be []:

personalize.delete_dataset(datasetArn=users_dataset_arn)
personalize.delete_dataset(datasetArn=products_dataset_arn)
personalize.delete_dataset(datasetArn=interactions_dataset_arn)
time.sleep(300)

datasets = personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']
if datasets:
    for dataset in datasets:
        print(dataset['name'])
else:
    print([])

In [ ]:
# Removing the dataset group - the output of this cell must be []:

personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)

dgs = personalize.list_dataset_groups()['datasetGroups']
available = []
for dg in dgs:
    if dg['name'] == dataset_group_arn:
        available.append(1)
if not available:
    print(available)

In [ ]:
# Removing the data schemas - the output of this cell must be []:

personalize.delete_schema(schemaArn='arn:aws:personalize:us-west-2:230440465708:schema/ret310-users-schema')
personalize.delete_schema(schemaArn='arn:aws:personalize:us-west-2:230440465708:schema/ret310-products-schema')
personalize.delete_schema(schemaArn='arn:aws:personalize:us-west-2:230440465708:schema/ret310-interactions-schema')

ret310_schemas = ['ret310-users-schema', 'ret310-products-schema', 'ret310-interactions-schema']

schemas = personalize.list_schemas()['schemas']
available = []
for schema in schemas:
    if schema['name'] in ret310_schemas:
        available.append(1)
if not available:
    print(available)

In [ ]:
# Deleting all IAM objects

iam.detach_role_policy(RoleName=role_name, PolicyArn=policyArn)
time.sleep(10)
iam.delete_policy(PolicyArn=policyArn)
time.sleep(10)
iam.delete_role(RoleName=role_name)